In [3]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, threshold=50, linewidth=75)

In [4]:
bikes_numpy = np.loadtxt(
    "../data/p1ch4/bike-sharing-dataset/hour-fixed.csv", 
    dtype=np.float32, 
    delimiter=",", 
    skiprows=1, 
    converters={1: lambda x: float(x[8:10])}
) # <1>

bikes = torch.from_numpy(bikes_numpy)
bikes, bikes.shape

(tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
         [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
         ...,
         [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
         [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]]),
 torch.Size([17520, 17]))

In [5]:
bikes.shape, bikes.stride(), bikes.shape[1]

(torch.Size([17520, 17]), (17, 1), 17)

In [6]:
# 第一个参数 -1 则原张量会变成一维的结构
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [7]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [21]:
first_day = bikes[:24].long()
print(first_day, first_day.shape)
weather_onehot = torch.zeros(first_day.shape[0], 4)
print("first_day:{}".format(first_day[:,9]))
print(weather_onehot, weather_onehot.shape)

tensor([[ 1,  1,  ..., 13, 16],
        [ 2,  1,  ..., 32, 40],
        ...,
        [23,  1,  ..., 17, 28],
        [24,  1,  ..., 24, 39]]) torch.Size([24, 17])
first_day:tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2,
        2, 2])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]) torch.Size([24, 4])


In [24]:
weather_onehot.scatter_(
    dim=1, 
    index=first_day[:,9].unsqueeze(1).long() - 1, # <1>
    value=1.0)

torch.Size([24, 4])

In [34]:
print(torch.cat((bikes[:24], weather_onehot), 1)[0])
# 1.0000,  0.0000,  0.0000,  0.0000] - 1
print(torch.cat((bikes[:24], weather_onehot), 1)[5])
# 0.0000, 1.0000, 0.0000, 0.0000 - 2
print(torch.cat((bikes[:24], weather_onehot), 1)[6])
# 1.0000, 0.0000, 0.0000, 0.0000 - 1

tensor([ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,
         6.0000,  0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,
         3.0000, 13.0000, 16.0000,  1.0000,  0.0000,  0.0000,  0.0000])
tensor([6.0000, 1.0000, 1.0000, 0.0000, 1.0000, 5.0000, 0.0000, 6.0000,
        0.0000, 2.0000, 0.2400, 0.2576, 0.7500, 0.0896, 0.0000, 1.0000,
        1.0000, 0.0000, 1.0000, 0.0000, 0.0000])
tensor([7.0000, 1.0000, 1.0000, 0.0000, 1.0000, 6.0000, 0.0000, 6.0000,
        0.0000, 1.0000, 0.2200, 0.2727, 0.8000, 0.0000, 2.0000, 0.0000,
        2.0000, 1.0000, 0.0000, 0.0000, 0.0000])


In [35]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape


torch.Size([730, 4, 24])

In [36]:
daily_weather_onehot.scatter_(
    1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [42]:
print("before: {}".format(daily_bikes.shape))
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)
# daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=0)
print("after: {}".format(daily_bikes.shape))

before: torch.Size([730, 29, 24])
after: torch.Size([730, 33, 24])


In [43]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0

In [13]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)
                         / (temp_max - temp_min))

In [14]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp))
                         / torch.std(temp))